In [130]:
#load and vis a few seqmented images in the images_CNN folder

import os
import glob
import h5py
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import Image, ImageSequence
img_list = glob.glob('images_CNN/F?_im.TIF')+glob.glob('images_CNN/F??_im.TIF')
img_list = sorted(img_list)
mask_list = glob.glob('images_CNN/*mask.h5')
mask_list = sorted(mask_list)
fluo_gfp = glob.glob('images_CNN/*GFP_im.TIF')
fluo_gfp = sorted(fluo_gfp)
fluo_rfp = glob.glob('images_CNN/*RFP_im.TIF')
fluo_rfp = sorted(fluo_rfp)
print( len(img_list), len(mask_list) , len(fluo_gfp), len(fluo_rfp  )   )
dataset = {}
for i in range(len(img_list)):
    dataset[i] = { 'img': img_list[i], 'mask': mask_list[i], 'gfp': fluo_gfp[i], 'rfp': fluo_rfp[i]  }
print(dataset)

11 11 11 11
{0: {'img': 'images_CNN/F10_im.TIF', 'mask': 'images_CNN/F10_mask.h5', 'gfp': 'images_CNN/F10_GFP_im.TIF', 'rfp': 'images_CNN/F10_RFP_im.TIF'}, 1: {'img': 'images_CNN/F11_im.TIF', 'mask': 'images_CNN/F11_mask.h5', 'gfp': 'images_CNN/F11_GFP_im.TIF', 'rfp': 'images_CNN/F11_RFP_im.TIF'}, 2: {'img': 'images_CNN/F1_im.TIF', 'mask': 'images_CNN/F1_mask.h5', 'gfp': 'images_CNN/F1_GFP_im.TIF', 'rfp': 'images_CNN/F1_RFP_im.TIF'}, 3: {'img': 'images_CNN/F2_im.TIF', 'mask': 'images_CNN/F2_mask.h5', 'gfp': 'images_CNN/F2_GFP_im.TIF', 'rfp': 'images_CNN/F2_RFP_im.TIF'}, 4: {'img': 'images_CNN/F3_im.TIF', 'mask': 'images_CNN/F3_mask.h5', 'gfp': 'images_CNN/F3_GFP_im.TIF', 'rfp': 'images_CNN/F3_RFP_im.TIF'}, 5: {'img': 'images_CNN/F4_im.TIF', 'mask': 'images_CNN/F4_mask.h5', 'gfp': 'images_CNN/F4_GFP_im.TIF', 'rfp': 'images_CNN/F4_RFP_im.TIF'}, 6: {'img': 'images_CNN/F5_im.TIF', 'mask': 'images_CNN/F5_mask.h5', 'gfp': 'images_CNN/F5_GFP_im.TIF', 'rfp': 'images_CNN/F5_RFP_im.TIF'}, 7: {'i

In [131]:
%load_ext autoreload
%autoreload 2


In [132]:

import skimage.measure as measure
import copy

def output_contours( m , cl , verbose = False):
    contours = measure.find_contours(m, .9)
    if verbose:
        plt.imshow(m)
        for n, contour in enumerate(contours):
            plt.plot(contour[:, 1], contour[:, 0], linewidth=2)
        plt.show()

    #output contours of each mask to file
    #divide x and y coordinates by total image size
    #to get values between 0 and 1  
    #<class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
    lines = []
    for c in contours:
        coords = []
        for i in range(0,c.shape[0],4):
            coords.append( float(c[i][0]) / m.shape[0] )
            coords.append( float(c[i][1]) / m.shape[1])
        line = str(cl) + ' ' + ' '.join([str(c) for c in coords]) + '\n'
        if verbose == True:
            print(line)
        lines.append(line)
    return lines

def split_mask(mask, crop = 1024):
    #custom encoding with 3 classes
    mask = mask[0:crop, 0:crop]
    mask1 = copy.deepcopy(mask)
    mask1[ (mask1 > 0) & (mask1 < 1000)] =  1
    mask1[mask1 > 1] = 0 

    mask2 = copy.deepcopy(mask)
    mask2[mask2 == 1] = 0
    mask2[ (mask2 > 1000) & (mask2 < 2000)] =  1
    mask2[mask2 > 1] = 0

    mask3 = copy.deepcopy(mask)
    mask3[mask3 == 1] = 0
    mask3[ (mask3 >= 2000) ] =  1
    mask3[mask3 > 1] = 0
    return mask1, mask2, mask3

def mask2contourfile( mask , outputfile , verbose = False):
    m1, m2, m3 = split_mask(mask)
    lines = output_contours(m1, 1 , verbose = verbose)
    lines += output_contours(m2, 2, verbose = verbose)
    lines += output_contours(m3, 3, verbose = verbose)
    
    with open(outputfile, 'w') as f:
        for l in lines:
            f.write(l)
    return  outputfile


In [133]:
#clean finaldataset folder
import shutil
overwite = True
if overwite:
    try:
        shutil.rmtree('./datasets/')
    except:
        pass
    os.mkdir('./datasets/')
    os.mkdir('./datasets/train')
    
    os.mkdir('./datasets/train/images/')
    os.mkdir('./datasets/train/labels/')
    os.mkdir('./datasets/val/')
    os.mkdir('./datasets/val/images')
    os.mkdir('./datasets/val/labels')


In [134]:
#stack equivalent frames together from img, gfp and rfp and transform them to a jpg image
import cv2
import numpy as np
import os
import tqdm

verbose = False

def yield_frames(img,crop=1024):
    for i, page in enumerate(ImageSequence.Iterator(img)):
        page = np.array(page)[0:crop, 0:crop]
        yield page
count = 0

for sample in dataset:
    maskfile = dataset[sample]['mask']
    maskh5 = h5py.File(maskfile, 'r')
    for group in maskh5.keys():
        for frame in maskh5[group]:
            mask = np.array( maskh5[group][frame] ,  dtype = np.uint16 )
            if np.sum(mask) > 0 :
                print( group, frame)
                print(np.unique(mask) )
                if verbose == True:
                    plt.imshow(mask)
                    plt.show()
                converted = mask2contourfile(mask, maskfile +'converted.txt' , verbose = verbose)
                dataset[sample]['mask_poly'] = converted
                break

    print('loading img')
    img = Image.open(dataset[sample]['img'])
    img = [frame for frame in yield_frames(img)]
    print('loading gfp')

    gfp = Image.open(dataset[sample]['gfp'])
    gfp = [frame for frame in yield_frames(gfp)]
    
    print('loading rfp')
    rfp = Image.open(dataset[sample]['rfp'])
    rfp = [frame for frame in yield_frames(rfp)]

    #stack the frames together
    for i in tqdm.tqdm(range(len(img))):
        im = np.stack([img[i], gfp[i], rfp[i]], axis=-1)
        im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
        cv2.imwrite('./datasets/train/images/img_'+str(count)+'.png', im )
        #save the correct mask file
        shutil.copyfile(dataset[sample]['mask_poly'], './datasets/train/labels/img_'+str(count)+'.txt' )
        count += 1

FOV0 T5
[   0    1    2    3    4 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1020 1021 1023 1024 1025 1026 1027 1028 1029 2001 2002 2003]
loading img
loading gfp
loading rfp


100%|██████████| 7/7 [00:00<00:00, 13.20it/s]


FOV0 T5
[   0 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 2001 2002]
loading img
loading gfp
loading rfp


100%|██████████| 7/7 [00:00<00:00, 13.35it/s]


FOV0 T5
[   0    1    2    3    4    5    6    7    8    9   10   11   12 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 1030 1031 1032 1033 1034 1035 1036 1037 1038 1039 1040 1041 1042 1043 1044 1045 1046 1047 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013]
loading img
loading gfp
loading rfp


...

100%|██████████| 7/7 [00:00<00:00, 13.30it/s]


FOV0 T5
[   0    1    2    3    4    5 1001 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 1030 1032 1033 1034 1035 1036 1037 2001 2002 2003]
loading img
loading gfp
loading rfp


100%|██████████| 7/7 [00:00<00:00, 13.05it/s]


FOV0 T5
[   0    1    2    3    4 1001 1002 1003 1004 1005 1006 1007 1008 1009 1011 1012 1013 1014 1015 1016 1018 1019 1021 1022 1023 1024 1025 1026 2001 2002 2003]
loading img
loading gfp
loading rfp


100%|██████████| 7/7 [00:00<00:00, 13.38it/s]


In [135]:
#move a fraction of the training data and corresponding labels to val
import random
import shutil
import os
files = os.listdir('./datasets/train/images/')
random.shuffle(files)
val_files = files[:int(len(files)*.1)]
for f in val_files:
    shutil.move('./datasets/train/images/'+f, './datasets/val/images/'+f)
    shutil.move('./datasets/train/labels/'+f.replace('.png', '.txt'), './datasets/val/labels/'+f.replace('.png', '.txt'))

In [136]:
#data augment the images and masks
#move files into train and test folders

In [146]:
#create traininging yaml file for the dataset

outyaml = """# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ./  # dataset root dir
train: train  
val: val  
test:  # test images (optional)

names: 
    0:'fused',
    1:'halted',
    2:'lysed'
"""

with open('./dataset.yaml', 'w') as f:
    f.write(outyaml)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-seg.pt")
results = model.train(
        batch=8,
        device="cpu",
        data='./dataset.yaml',
        epochs=3,
        imgsz=1024,
    )

Ultralytics YOLOv8.1.9 🚀 Python-3.10.8 torch-1.13.1 CPU (AMD EPYC 7443 24-Core Processor)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=./dataset.yaml, epochs=3, time=None, patience=50, batch=8, imgsz=1024, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train38, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False

train: Scanning /work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/segmentation_fluo/datasets/train/labels.cache... 70 images, 0 backgrounds, 0 corrupt: 100%|██████████| 70/70 [00:00<?, ?it/s]
val: Scanning /work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/segmentation_fluo/datasets/val/labels.cache... 7 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<?, ?it/s]

Plotting labels to runs/segment/train38/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000278, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 0 dataloader workers
Logging results to runs/segment/train38
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


        1/3         0G       3.77      6.037      5.629      2.888        329       1024:  44%|████▍     | 4/9 [01:00<01:16, 15.21s/it]

In [ ]:
from IPython.display import Image as show_image
show_image(filename="runs/segment/train60/val_batch0_labels.jpg")

show_image(filename="runs/segment/train60/MaskP_curve.png")

show_image(filename="runs/segment/train60/results.png")



In [ ]:
#image augmentation for training

import cv2
import random
import numpy as np

def random_rotation(image, mask, angle_range):
    angle = random.uniform(-angle_range, angle_range)
    image = Image.fromarray(image)
    mask = Image.fromarray(mask)
    image = image.rotate(angle)
    mask = mask.rotate(angle)
    return np.array(image), np.array(mask)

def random_flip(image, mask):
    if random.random() > 0.5:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)
    return image, mask

def random_crop(image, mask, crop_size):
    h, w, _ = image.shape
    crop_h, crop_w = crop_size
    top = np.random.randint(0, h - crop_h)
    left = np.random.randint(0, w - crop_w)
    bottom = top + crop_h
    right = left + crop_w
    image = image[top:bottom, left:right]
    mask = mask[top:bottom, left:right]
    return image, mask

def random_augmentation(image, mask, angle_range, crop_size):
    image, mask = random_rotation(image, mask, angle_range)
    image, mask = random_flip(image, mask)
    image, mask = random_crop(image, mask, crop_size)
    return image, mask

#resize to original size
def resize(image, mask, size):
    image = cv2.resize(image, size)
    mask = cv2.resize(mask, size)
    return image, mask

#apply augmentation and then resize to original size
def augment_and_resize(image, mask, angle_range, crop_size, size):
    image, mask = random_augmentation(image, mask, angle_range, crop_size)
    image, mask = resize(image, mask, size)
    return image, mask



frames	framee	cuttop	cutbttm	cutleft	cutrght
0	6	1	1040	1	1392



In [ ]:
#augment dataset and dump to disk in pt format


In [ ]:
#retrain yolo segmentation net on the new data

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img = Image.open(self.dataset[idx]['img'])
        maskfile = h5py.File(self.dataset[idx]['mask'], 'r')
        
        for group in maskfile:
            for frame in maskfile[group]:
                mask = np.array( maskfile[group][frame] ,  dtype = np.uint16 )
                if np.sum(mask) > 0 :
                    mask = mask
        
        if self.transform:
            img = self.transform(img)
        
        return img, mask

transform = transforms.Compose([ transforms.ToTensor() ])
dataset = CustomDataset(dataset, transform=transform)

train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#traininig loop
num_epochs = 10
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

In [ ]:
#predict on the test set
model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        print(outputs.shape, targets.shape)
        break


#visualize the results
import matplotlib.pyplot as plt
import numpy as np
import torchvision

for i in range(2):
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(np.array(inputs[i].permute(1, 2, 0)))
    plt.title('Input')
    plt.subplot(1, 2, 2)
    plt.imshow(targets[i].squeeze(), cmap='gray')
    plt.title('Target')
    plt.show()
    plt.figure(figsize=(20, 10))
    plt.imshow(outputs[i].squeeze(), cmap='gray')
    plt.title('Output')
    plt.show()
    break